In [139]:
import sys
import argparse
import os
import pandas as pd
import datetime as dt
from datetime import timedelta
from datetime import datetime
import numpy as np
import json
import geopandas as gpd
from urllib.request import urlopen

In [140]:
world_deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
world_deaths.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/8/21,12/9/21,12/10/21,12/11/21,12/12/21,12/13/21,12/14/21,12/15/21,12/16/21,12/17/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7317,7321,7322,7325,7328,7328,7329,7331,7332,7332
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3126,3128,3130,3132,3134,3140,3144,3152,3156,3158
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6122,6126,6132,6137,6145,6151,6155,6165,6171,6175
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,133,133,133,133,133,133,134,134,134,134
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,1735,1736,1737,1737,1737,1737,1737,1737,1737,1738


In [141]:
print(world_deaths.columns)

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '12/8/21', '12/9/21', '12/10/21', '12/11/21', '12/12/21', '12/13/21',
       '12/14/21', '12/15/21', '12/16/21', '12/17/21'],
      dtype='object', length=700)


In [142]:
# changing the columns to match the US aggregated data
world_deaths = world_deaths.drop(['Province/State', 'Lat', 'Long'], axis = 1)

# format the dates
dic = {}
for i in range(len(world_deaths.columns)):
    to_replace = list(world_deaths.columns)[i].replace('/', '-')
    year = to_replace[-2:]  
    year = "20" + year
    to_replace = to_replace[:len(to_replace) - 3]
    
    to_replace = year + '-' + to_replace
    
    dic[list(world_deaths.columns)[i]] = to_replace
    

world_deaths = world_deaths.rename(index = str, columns = dic)
world_deaths = world_deaths.rename(columns = {"20on-Country-Reg": "name"})
world_deaths.head()

,name,2020-1-22,2020-1-23,2020-1-24,2020-1-25,2020-1-26,2020-1-27,2020-1-28,2020-1-29,2020-1-30,...,2021-12-8,2021-12-9,2021-12-10,2021-12-11,2021-12-12,2021-12-13,2021-12-14,2021-12-15,2021-12-16,2021-12-17
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,7317,7321,7322,7325,7328,7328,7329,7331,7332,7332
1,Albania,0,0,0,0,0,0,0,0,0,...,3126,3128,3130,3132,3134,3140,3144,3152,3156,3158
2,Algeria,0,0,0,0,0,0,0,0,0,...,6122,6126,6132,6137,6145,6151,6155,6165,6171,6175
3,Andorra,0,0,0,0,0,0,0,0,0,...,133,133,133,133,133,133,134,134,134,134
4,Angola,0,0,0,0,0,0,0,0,0,...,1735,1736,1737,1737,1737,1737,1737,1737,1737,1738


In [143]:
# filter out the dates not needed
worldStart = dt.datetime(2020, 1, 22)
startDate = dt.datetime(2020, 2, 14)
endDate = dt.datetime(2020, 4, 15)
# print(int((endDate - startDate).days))

# datetime_columns = pd.to_datetime(world_deaths.columns[1:])
# column_drop_list = [i for i in list(world_case.columns) if date]


In [144]:
len(world_deaths)

280

In [145]:
# group by name
world_deaths = pd.DataFrame(world_deaths.groupby(by = ["name"], as_index = False).sum())
world_deaths.head()

,name,2020-1-22,2020-1-23,2020-1-24,2020-1-25,2020-1-26,2020-1-27,2020-1-28,2020-1-29,2020-1-30,...,2021-12-8,2021-12-9,2021-12-10,2021-12-11,2021-12-12,2021-12-13,2021-12-14,2021-12-15,2021-12-16,2021-12-17
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,7317,7321,7322,7325,7328,7328,7329,7331,7332,7332
1,Albania,0,0,0,0,0,0,0,0,0,...,3126,3128,3130,3132,3134,3140,3144,3152,3156,3158
2,Algeria,0,0,0,0,0,0,0,0,0,...,6122,6126,6132,6137,6145,6151,6155,6165,6171,6175
3,Andorra,0,0,0,0,0,0,0,0,0,...,133,133,133,133,133,133,134,134,134,134
4,Angola,0,0,0,0,0,0,0,0,0,...,1735,1736,1737,1737,1737,1737,1737,1737,1737,1738


In [146]:
len(world_deaths)

196

In [147]:
# convert cumulative to daily
for i in range(len(world_deaths.columns) - 1, 2, -1):
    world_deaths[world_deaths.columns[i]] = world_deaths[world_deaths.columns[i]].sub(world_deaths[world_deaths.columns[i - 1]], axis = 0)
# world_deaths.head()

In [148]:
world_deaths = world_deaths.drop(world_deaths.columns[1:(1+int((startDate - worldStart).days))], axis = 1)
world_deaths = world_deaths.drop(world_deaths.columns[(1+int((endDate - startDate).days)):], axis = 1)
world_deaths.head()

,name,2020-2-14,2020-2-15,2020-2-16,2020-2-17,2020-2-18,2020-2-19,2020-2-20,2020-2-21,2020-2-22,...,2020-4-5,2020-4-6,2020-4-7,2020-4-8,2020-4-9,2020-4-10,2020-4-11,2020-4-12,2020-4-13,2020-4-14
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,2,0,4,3,1,0,0,3,1,3
1,Albania,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,0,0,0,1
2,Algeria,0,0,0,0,0,0,0,0,0,...,22,21,20,12,30,21,19,18,20,13
3,Andorra,0,0,0,0,0,0,0,0,0,...,1,3,1,1,2,1,0,3,0,2
4,Angola,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [149]:
# merge with the data with geoid
world_geoid = pd.read_csv('data/georegion.csv')
print(world_geoid.columns)
world_deaths = world_geoid.merge(world_deaths, left_on = world_geoid.columns[0], right_on = "name", how = 'left')


# to_drop = []
# to_drop.append(len(world_deaths.columns) - 1)
# for i in range(2, 25):
#     to_drop.append(i)
# world_deaths = world_deaths.drop(world_deaths.columns[to_drop], axis = 1)

world_deaths = world_deaths.drop(world_deaths.columns[2], axis = 1)
world_deaths.head()

Index(['﻿geoName', 'geoRegion'], dtype='object')


,﻿geoName,geoRegion,2020-2-14,2020-2-15,2020-2-16,2020-2-17,2020-2-18,2020-2-19,2020-2-20,2020-2-21,...,2020-4-5,2020-4-6,2020-4-7,2020-4-8,2020-4-9,2020-4-10,2020-4-11,2020-4-12,2020-4-13,2020-4-14
0,Antigua and Barbuda,NORTH AMERICA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Algeria,AFRICA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,22.0,21.0,20.0,12.0,30.0,21.0,19.0,18.0,20.0,13.0
2,Azerbaijan,ASIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
3,Albania,EURPOE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,Armenia,ASIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,2.0,1.0,0.0,1.0,2.0


In [150]:
len(world_deaths)

238

In [151]:
# world_deaths[world_deaths['_merge'] != 'both']

In [152]:
len(world_deaths)

238

In [153]:
output_deaths = pd.read_csv("data/output_deaths.csv")
output_deaths.head()


# world_deaths
# world_deaths.to_csv("data/world_deaths.csv", index = False)

,name,state,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,...,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14
0,Phoenix-Mesa-Scottsdale,AZ,0,0,0,0,0,0,0,0,...,3,0,4,2,5,2,6,2,4,3
1,San Jose-Sunnyvale-Santa Clara,CA,0,0,0,0,0,0,0,0,...,0,0,4,3,1,2,2,2,7,0
2,San Francisco-Oakland-Hayward,CA,0,0,0,0,0,0,0,0,...,1,5,11,2,2,7,3,3,1,1
3,San Diego-Carlsbad,CA,0,0,0,0,0,0,0,0,...,1,0,12,5,4,4,1,0,2,6
4,Los Angeles-Long Beach-Anaheim,CA,0,0,0,0,0,0,0,0,...,15,15,23,31,25,18,25,32,24,40


In [154]:
# world_deaths = pd.concat([output_deaths, world_deaths], axis = 0)
world_deaths.columns


Index(['﻿geoName', 'geoRegion', '2020-2-14', '2020-2-15', '2020-2-16',
       '2020-2-17', '2020-2-18', '2020-2-19', '2020-2-20', '2020-2-21',
       '2020-2-22', '2020-2-23', '2020-2-24', '2020-2-25', '2020-2-26',
       '2020-2-27', '2020-2-28', '2020-2-29', '2020-3-1', '2020-3-2',
       '2020-3-3', '2020-3-4', '2020-3-5', '2020-3-6', '2020-3-7', '2020-3-8',
       '2020-3-9', '2020-3-10', '2020-3-11', '2020-3-12', '2020-3-13',
       '2020-3-14', '2020-3-15', '2020-3-16', '2020-3-17', '2020-3-18',
       '2020-3-19', '2020-3-20', '2020-3-21', '2020-3-22', '2020-3-23',
       '2020-3-24', '2020-3-25', '2020-3-26', '2020-3-27', '2020-3-28',
       '2020-3-29', '2020-3-30', '2020-3-31', '2020-4-1', '2020-4-2',
       '2020-4-3', '2020-4-4', '2020-4-5', '2020-4-6', '2020-4-7', '2020-4-8',
       '2020-4-9', '2020-4-10', '2020-4-11', '2020-4-12', '2020-4-13',
       '2020-4-14'],
      dtype='object')

In [155]:
output_deaths.columns

Index(['name', 'state', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17',
       '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22',
       '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27',
       '2020-02-28', '2020-02-29', '2020-03-01', '2020-03-02', '2020-03-03',
       '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08',
       '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13',
       '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17', '2020-03-18',
       '2020-03-19', '2020-03-20', '2020-03-21', '2020-03-22', '2020-03-23',
       '2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27', '2020-03-28',
       '2020-03-29', '2020-03-30', '2020-03-31', '2020-04-01', '2020-04-02',
       '2020-04-03', '2020-04-04', '2020-04-05', '2020-04-06', '2020-04-07',
       '2020-04-08', '2020-04-09', '2020-04-10', '2020-04-11', '2020-04-12',
       '2020-04-13', '2020-04-14'],
      dtype='object')

In [156]:
rename_col = {}
for i in range(len(output_deaths.columns)):
    rename_col[list(output_deaths.columns)[i]] = list(world_deaths.columns)[i]
output_deaths = output_deaths.rename(index = str, columns = rename_col)
# print(rename_col)
# print(output_deaths.columns == world_deaths.columns)

In [157]:
world_deaths = world_deaths.append(output_deaths, ignore_index = True)

In [158]:
world_deaths.head()

,﻿geoName,geoRegion,2020-2-14,2020-2-15,2020-2-16,2020-2-17,2020-2-18,2020-2-19,2020-2-20,2020-2-21,...,2020-4-5,2020-4-6,2020-4-7,2020-4-8,2020-4-9,2020-4-10,2020-4-11,2020-4-12,2020-4-13,2020-4-14
0,Antigua and Barbuda,NORTH AMERICA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Algeria,AFRICA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,22.0,21.0,20.0,12.0,30.0,21.0,19.0,18.0,20.0,13.0
2,Azerbaijan,ASIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
3,Albania,EURPOE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,Armenia,ASIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,2.0,1.0,0.0,1.0,2.0


In [159]:
georegion_af = pd.read_csv("./data/georegion_af.csv")

In [160]:
georegion_af = georegion_af[georegion_af['GEOID'] != -9999]

In [161]:
georegion_af.head()

,GEOID,geoName,geoRegion,geoLON,geoLAT,geoISO3
1,1,Antigua and Barbuda,NORTH AMERICA,-61.791,17.280,ATG
2,2,Algeria,AFRICA,2.632,28.163,DZA
3,3,Azerbaijan,ASIA,47.532,40.292,AZE
4,4,Albania,EURPOE,20.068,41.143,ALB
5,5,Armenia,ASIA,44.948,40.286,ARM


In [162]:
len(georegion_af)

1168

In [163]:
world_deaths.columns

Index(['﻿geoName', 'geoRegion', '2020-2-14', '2020-2-15', '2020-2-16',
       '2020-2-17', '2020-2-18', '2020-2-19', '2020-2-20', '2020-2-21',
       '2020-2-22', '2020-2-23', '2020-2-24', '2020-2-25', '2020-2-26',
       '2020-2-27', '2020-2-28', '2020-2-29', '2020-3-1', '2020-3-2',
       '2020-3-3', '2020-3-4', '2020-3-5', '2020-3-6', '2020-3-7', '2020-3-8',
       '2020-3-9', '2020-3-10', '2020-3-11', '2020-3-12', '2020-3-13',
       '2020-3-14', '2020-3-15', '2020-3-16', '2020-3-17', '2020-3-18',
       '2020-3-19', '2020-3-20', '2020-3-21', '2020-3-22', '2020-3-23',
       '2020-3-24', '2020-3-25', '2020-3-26', '2020-3-27', '2020-3-28',
       '2020-3-29', '2020-3-30', '2020-3-31', '2020-4-1', '2020-4-2',
       '2020-4-3', '2020-4-4', '2020-4-5', '2020-4-6', '2020-4-7', '2020-4-8',
       '2020-4-9', '2020-4-10', '2020-4-11', '2020-4-12', '2020-4-13',
       '2020-4-14'],
      dtype='object')

In [164]:
world_deaths = world_deaths.merge(georegion_af, left_on = [world_deaths.columns[0], "geoRegion"] , right_on = ["geoName", "geoRegion"], how = 'right')

In [165]:
# reorder the columns
columns_reordered1 = list(world_deaths.columns)[1:2]
columns_reordered2 = list(world_deaths.columns)[2:-6]
columns_reordered3 = list(world_deaths.columns)[-5:-3]
columns_reordered4 = list(world_deaths.columns)[-3:]
columns_reordered = columns_reordered3 + columns_reordered1 + columns_reordered4 + columns_reordered2 


world_deaths = world_deaths.reindex(columns = columns_reordered)

In [166]:
# change the floats to integers
def float_to_int(x):
    if (x != x):
         # print("hey")
        return -9999
    x = int(x)
    return x

for i in list(world_deaths.columns[6:]):
    world_deaths[i] = world_deaths[i].apply(float_to_int)

def format_float(x):
    x = round(x, 3)
    return x

world_deaths['geoLON'] = world_deaths['geoLON'].apply(format_float)
world_deaths['geoLAT'] = world_deaths['geoLAT'].apply(format_float)
    

In [167]:
world_deaths.head()

,GEOID,geoName,geoRegion,geoLON,geoLAT,geoISO3,2020-2-14,2020-2-15,2020-2-16,2020-2-17,...,2020-4-4,2020-4-5,2020-4-6,2020-4-7,2020-4-8,2020-4-9,2020-4-10,2020-4-11,2020-4-12,2020-4-13
0,1,Antigua and Barbuda,NORTH AMERICA,-61.791,17.280,ATG,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
1,2,Algeria,AFRICA,2.632,28.163,DZA,0,0,0,0,...,25,22,21,20,12,30,21,19,18,20
2,3,Azerbaijan,ASIA,47.532,40.292,AZE,0,0,0,0,...,0,2,0,1,0,1,1,1,0,1
3,4,Albania,EURPOE,20.068,41.143,ALB,0,0,0,0,...,3,0,1,1,0,1,0,0,0,0
4,5,Armenia,ASIA,44.948,40.286,ARM,0,0,0,0,...,0,0,1,0,1,1,2,1,0,1


In [168]:
len(world_deaths)

1168

In [169]:
world_deaths.to_csv("data/world_deaths.csv", index = False)